Привет! Меня зовут Александр Арасланов, я провожу ревью твоей работы. Предалагю общаться на «ты», если не против.

В работе ты увидишь комментарии, помеченные разным цветом.

<div class="alert alert-danger">
Красным цветом помечены моменты, которые необходимо доработать, чтобы проект был принят. Уверен, что у тебя все получится. Буду ждать твою работу на повторное ревью. 
</div>

<div class="alert alert-success">
Зеленым цветом помечены оброзцовые решения. Которые можно использовать в дальнейшем.
</div>

<div class="alert alert-warning">
Желтым цветом отмечено то, что ты можешь улучшить или доработать. Можешь учесть это при выполнении следующих работ или доработать проект сейчас, но это не обязательно.
</div>


Будет комфортнее работать над проектом вместе, если ты будешь **писать об изменениях** в проекте по моим рекомендациям. Для этого выбери для своих комментариев 

<div class="alert alert-block alert-info">заметный цвет</div>

Пожалуйста, **не перемещай, не изменяй и не удаляй мои комментарии**. Всё это поможет выполнить повторную проверку твоего проекта оперативнее.

Дарья, вижу, что старалсь. Хорошо справилась с большинством моментов, не хватило внимательности в некторых расчтеах. Пока не могу принять проект, внеси доработки и присылай его на повторную проверку.

___

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
# просмотр первых десяти строчек в таблице data
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
# получение общей информации о данных в таблице data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# проверим правильно ли записаны названия столбцов таблицы data
display(data.columns.to_list())

['children',
 'days_employed',
 'dob_years',
 'education',
 'education_id',
 'family_status',
 'family_status_id',
 'gender',
 'income_type',
 'debt',
 'total_income',
 'purpose']

**Вывод**

Мной были обнаружены ошибки в данных: 1) в days_employed, отвечающий за общий трудовой стаж в днях, встречаются отрицательные значения, отрицательный стаж невозможен 2) в education встречаются дубликаты, например, высшее и ВЫСШЕЕ

В days_employed и total_income есть пропуски в данных. Вряд ли клиенты не заполнили эти данные, ведь без данных о доходе и постоянной работы не получить кредит. Скорее всего есть какая-то ошибка в записи данных.

В названии столбцов проблем нет

## Шаг 2. Предобработка данных

### Обработка отрицательных значений

In [29]:
# возьмем те же значения но по модулю
data['days_employed'] = abs(data['days_employed'])
data['children'] = abs(data['children'])
# проверка выводом первых 10 строк
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,высокий
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,средний
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,средний
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,средний


**Вывод**

Так как у нас нет возможности узнать, почему встречаются отрицательные значения в таком большом количестве, просто возьмем эти значения по модулю.

### Обработка пропусков

In [5]:
# подсчёт пропусков
display(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
# замена всех пропусков
data = data.dropna(subset = ['days_employed'])
average_income = data['total_income'].mean()
data = data.fillna(average_income)

<div class="alert alert-warning">

**Комментарий ревьюера**

Доход лучше было бы заменить на среднее значение</div>

<div class="alert alert-warning" style="background:#ffdbf1;color:#2e00ab">

**Комментарий студента**
    
Да, согласна, что так логичней, не догадалась, исправила</div>

In [7]:
# повторный подсчёт пропусков
display(data.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Пропуски были в days_employed и total_income. Как я уже сказала выше, надо сообщить об этом. Данные из столбца days_employed вообще не нужны для решения поставленного вопроса, поэтому я решила убрать пропуски из этого столбца, а данные из стобца total_income нужны, поэтому я заменила их на 0

### Замена типа данных

In [8]:
# столбцы, значения которых нужно перевести в int64
columns = ['days_employed', 'total_income']
# перевод значений в int64
for column in columns:
    data[column] = data[column].astype('int')

In [9]:
# поверим успешность перевода
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null int64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null int64
purpose             19351 non-null object
dtypes: int64(7), object(5)
memory usage: 1.9+ MB


**Вывод**

У нас есть два знакомых инструмента для перевода одного типа данных в другой: to_numeric и astype. to_numeric переводит строку в вещественное или целое число, но так как нам нужно было перевести вещественные числа в целые, этот вариант нам не подходит. astype переводит из любого типа данных в указанный нами в скобках, поэтому я выбрала его.

### Обработка дубликатов

In [10]:
# дубликаты могут быть явными и неявными, начнем, с явных
# подсчёт явных дубликатов
print(data.duplicated().sum())

0


In [11]:
# теперь неявные
# просмотр уникальных названий столбца с данными о семейном статусе
family_status_column = data['family_status']
family_status_column = family_status_column.sort_values()
display(family_status_column.unique())

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

In [12]:
# просмотр уникальных названий столбца с данными о должности
income_type_column = data['income_type']
income_type_column = income_type_column.sort_values()
display(income_type_column.unique())

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

In [13]:
# просмотр уникальных названий столбца с данными об образовании
education_column = data['education']
education_column = education_column.sort_values()
display(education_column.unique())

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

In [14]:
# все буквы становятся строчными
data['education'] = data['education'].str.lower()

In [15]:
# проверка, что все успешно
education_column_lowercase = data['education']
education_column_lowercase = education_column_lowercase.sort_values()
display(education_column_lowercase.unique())

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

<div class="alert alert-warning">

**Комментарий ревьюера**

После обработки неявных дубликатов обязательно проверяй на появление полных дубликатов, потому что они могут появиться</div>

<div class="alert alert-warning" style="background:#ffdbf1;color:#2e00ab">

**Комментарий студента**
    
Не знала об этом</div>

In [16]:
# повторная проверка на неявные дубликаты
print(data.duplicated().sum())

0


In [17]:
# просмотр уникальных названий столбца с данными о цели кредита
purpose_column = data['purpose']
purpose_column = purpose_column.sort_values()
display(purpose_column.unique())

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

**Вывод**

Явных дубликатов не оказалось. В столбцам с данными о семейном статусе и о должности неявных дубликатов не было обнаружено. В столбце с данными об образовании были неявные дубликаты в связи с разным регистром в записи одинаковых слов, полагаю, это случилось из-за человеческой ошибки, но ошибки были успешно исправлены с помощью str.lower. В столбце с данными о цели кредита много неявных дубликатов, их исправим с помощью лемматизации.

### Лемматизация

In [18]:
# подключение библиотеки для лемматизации
from pymystem3 import Mystem
m = Mystem()
# строка с уникальными значениями из столбца purpose
unique_purpose = purpose_column.unique()
# вывод лемм
for purpose in unique_purpose:
    print(m.lemmatize(purpose))

['автомобиль', '\n']
['автомобиль', '\n']
['высокий', ' ', 'образование', '\n']
['дополнительный', ' ', 'образование', '\n']
['жилье', '\n']
['заниматься', ' ', 'высокий', ' ', 'образование', '\n']
['заниматься', ' ', 'образование', '\n']
['на', ' ', 'покупка', ' ', 'автомобиль', '\n']
['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n']
['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n']
['на', ' ', 'проведение', ' ', 'свадьба', '\n']
['недвижимость', '\n']
['образование', '\n']
['операция', ' ', 'с', ' ', 'жилье', '\n']
['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['операция', ' ', 'с', ' ', 'недвижимость', '\n']
['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n']
['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n']
['покупка', ' ', 'жилье', '\n']
['покупка', ' ', 'жилье', ' ', 'для', ' ', 'сдача', '\n']
['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n']
['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['по

In [19]:
# функция для создания "новых" целей по леммам
def make_category(purpose):
    lemmas = m.lemmatize(purpose)
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    elif 'образование' in lemmas:
        return 'образование'
    elif 'недвижимость' in lemmas or 'жилье' in lemmas:
        return 'недвижимость'

In [20]:
# применение функции к всем значениям столбца purpose и запись столбца category
data['category'] = data['purpose'].apply(make_category)
# вывод первых десяти строк для проверки
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


**Вывод**

Разделив все на леммы с помощью библиотеки Mystem, я заметила, что в каждой строке встречается либо "автомобиль", либо "образование", либо "свадьба", либо "недвижимость", либо "жилье". Именно по этим словам я потом создавала целы

### Категоризация данных

In [21]:
# функция для категоризации по доходу
def income_group(income):
    if income < 60000:
        return 'низкий'
    elif income < 200000:
        return 'средний'
    elif income > 200000:
        return 'высокий'

In [28]:
# применение функции ко всем значениям в столбце total_income
data['income_group'] = data['total_income'].apply(income_group)

<div class="alert alert-danger">

**Комментарий ревьюера**

Нужно было выделит категории например по доходу, т.е разделить все данные на низкий/средний/высокий. Также можешь выделить категории наличия одного, нескольких или их отсутсвия </div>

<div class="alert alert-warning" style="background:#ffdbf1;color:#2e00ab">

**Комментарий студента**
    
А, поняла</div>

<div class="alert alert-success">

**Комментарий ревьюераV2**

Все отлично!</div>

**Вывод**

Я разделила на категории по доходу - низкий, средний и высокий. Низкий - это меньше 60 тысяч, средний - от 60 до 200 тысяч, высокий - больше 200 тысяч.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

<div class="alert alert-danger">

**Комментарий ревьюера**

Дарья, здесь ошибка в выборе столбца, правльно будет использовать столбец, отвечающий за наличие детей.</div>

<div class="alert alert-warning" style="background:#ffdbf1;color:#2e00ab">

**Комментарий студента**
    
Перепутала столбцы</div>

<div class="alert alert-success">

**Комментарий ревьюераV2**

Исправления приняты</div>

In [30]:
display(data.groupby('children')['debt'].mean())

children
0     0.074902
1     0.093230
2     0.095624
3     0.074830
4     0.088235
5     0.000000
20    0.119403
Name: debt, dtype: float64

**Вывод**

Из таблицы мы видим, что хуже всех выплачивают кредит люди с двумя детьми или одним ребенком, лучше всех - с 3 детьми и вообще без детей. Никакой зависимости нет.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
display(data.groupby('family_status')['debt'].mean())

family_status
Не женат / не замужем    0.100594
в разводе                0.070175
вдовец / вдова           0.064740
гражданский брак         0.090763
женат / замужем          0.075922
Name: debt, dtype: float64

<div class="alert alert-success">

**Комментарий ревьюера**

Молодец, здесь все сделано верно. </div>

**Вывод**

Обнаружена зависимость, что лучше всех выплачивают задолженность люди, бывшие в браке (в разводе и вдовы/вдовцы), немного хуже выплачивают люди в браке (женат/замужем и гражданский брак), а хуже всех - не женат/не замужем.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
display(data.groupby('income_group')['debt'].mean())

income_group
высокий    0.070667
низкий     0.060794
средний    0.086357
Name: debt, dtype: float64

<div class="alert alert-danger">

**Комментарий ревьюера**

Во время категоризации необходимо было выделить категории дохода (выский/средний/низкий), сгруппировать по ним и найти долю должников </div>

<div class="alert alert-warning" style="background:#ffdbf1;color:#2e00ab">

**Комментарий студента**
    
Поправила</div>

<div class="alert alert-success">

**Комментарий ревьюераV2**

Исправления приняты</div>

**Вывод**

Интуитивно, казалось, что чем выше доход, тем лучше человек должен выплачивать долг. Но оказалось, что самый надежный клиент - со средним доходом, а самый ненадежный - с низким.

- Как разные цели кредита влияют на его возврат в срок?

In [27]:
display(data.groupby('category')['debt'].mean())

category
автомобиль      0.094175
недвижимость    0.073273
образование     0.092021
свадьба         0.075274
Name: debt, dtype: float64

<div class="alert alert-success">

**Комментарий ревьюера**

Все верно👍</div>

**Вывод**

Из таблицы видно, что лучше всего выплачивают задолженности на недвижимость и свадьбу, а хуже всего на автомобиль и  образование.

## Шаг 4. Общий вывод

На выплачивание кредитов вовремя влиет семейное положение, доход и цели кредита. Люди, бывшие в браке (в разводе и вдовы/вдовцы), и люди, находящиеся в браке (женат/замужем и гражданский брак), более надежны для банка, чем не замужние и не женатые люди. Однако, количество детей не влияет на выплачивание долгов. Доход был разделен на три категории: низкий, средний и высокий. Люди со средним доходом показывют самую большую надежность для банка, после идут люди с высоким доходом, а самые ненадежные - люди с низким доходом. Люди берут кредит по 4 причинам: автомобиль, свадьба, недвижимость, образование. Лучше всего выплачивают задолженности на недвижимость и свадьбу, а хуже всего на автомобиль и образование.

<div class="alert alert-danger">

**Комментарий ревьюера**

Молодец, хорошо справилась с проектом. Есть структура, хорошая подготовка данных. Но нсть ошибки в расчетах. Не забудь пересмотреть выводы, промежуточный и общий. Рекомендую сделать общий вывод более детальным, например, расскажи про то, какие категории удалось выделить в причинах выдачи кредита или уровня дохода</div>

<div class="alert alert-warning" style="background:#ffdbf1;color:#2e00ab">

**Комментарий студента**
    
Спасибо, исправила</div>

<div class="alert alert-success">

**Комментарий ревьюераV2**

Испрвыления принял. Молодец👍 Успехов в дальнейшей учебе!</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.